# NCSU GIS 582: Geospatial Modeling and Analysis

## 7B: Hydrologic and Erosion modeling

### Outline:

- geospatial aspects of models: spatially averaged and distributed models
- spatial hydrologic modeling
- spatial modeling of erosion and sediment transport
- deriving input parameters, analysis and visualization of modeling results
- modeling impact of landuse change

### Lecture:
[Lecture slides: Hydrologic and Erosion Modeling](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/lectures/hydro_erosion.html)

### Supplemental materials:

* [Surface Water Flow Modeling](https://link-springer-com.prox.lib.ncsu.edu/chapter/10.1007/978-3-319-89303-7_7) and [Soil Erosion Modeling](https://link-springer-com.prox.lib.ncsu.edu/chapter/10.1007/978-3-319-89303-7_8) chapters from Petrasova et al., 2018, Tangible Modeling with Open Source GIS. Second Edition. Springer. [Available free through NCSU library](https://link-springer-com.prox.lib.ncsu.edu/book/10.1007/978-3-319-89303-7). or through [Springer](https://link.springer.com/book/10.1007%2F978-3-319-89303-7)
* [Mitasova et al., 2013, GIS-based soil erosion modeling](https://www-sciencedirect-com.prox.lib.ncsu.edu/science/article/pii/B978012374739600052X), In: Treatise on Geomorphology. Academic Press, San Diego, CA, vol. 3, Remote Sensing and GIScience in Geomorphology, pp. 228–258
* [Soil erosion and deposition modeling tutorial](http://ncsu-geoforall-lab.github.io/erosion-modeling-tutorial/) (material under development)
* [Erosion modeling tutorial](http://www4.ncsu.edu/~hmitaso/gmslab/reports/CerlErosionTutorial/denix/denixstart.html) (old but useful document)

### Resources:

- [GRASS GIS overview and manual](https://grass.osgeo.org/grass83/manuals/index.html)
- [Recommendations](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data_acquisition.html#commands) and [tutorial](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/grass_intro.html) how to use GUI from the first assignment
- [Brief theoretical background, equations, units](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/resources/erosion_notes.pdf)
- [C-factor values](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/cfactor.txt)
- [K-factor](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/resources/kfactor.html)
- [R-factor](https://www.ars.usda.gov/ARSUserFiles/64080530/RUSLE/AH_703.pdf) (page 47)

Input Text files with recode rules and color rules:

- Land use category description (lu_labels.txt)
- C-factor recode table (cfac_rules.txt)
- C-factor color table (cfac_color.txt)
- Soil loss color table (soilloss_color.txt)
- Erosion/deposition color table (erdep_color.txt)
- Erosion/deposition classes (erdep_class.txt)
- Erosion/deposition class labels (erdep_label.txt)


In [ ]:
import sys
v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

## Start GRASS GIS

Create a new mapset to run your analysis.

In [ ]:
%%bash

grass -c -e ~/grassdata/nc_spm_08_grass7/HW_7B_Flow_Modeling

In [ ]:
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
# FOR WINDOWS:
# grass_call = "grass80"
# shell_setting = True
# FOR MAC/LINUX
grass_call = "grass"
shell_setting = False

sys.path.append(
    subprocess.check_output([grass_call, "--config", "python_path"], text=True, shell=shell_setting).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata", "nc_spm_08_grass7", "HW_7B_Flow_Modeling")

## Compute soil detachment using USLE3D

**Compute topographic potential (LS factor)**

Compare impact of the power function exponents on the erosion pattern.

In [ ]:
%%bash

g.region raster=elev_lid792_1m -p
r.slope.aspect elevation=elev_lid792_1m slope=slope_1m aspect=aspect_1m
r.flow elevation=elev_lid792_1m flowaccumulation=flowacc_1m
r.mapcalc "lsfac3d_1m = 1.2 * pow(flowacc_1m * 1./22.1,0.2) * pow(sin(slope_1m)/0.09,1.3)"
r.mapcalc "lsfac3d_s1_1m = 1.5 * pow(flowacc_1m * 1./22.1,0.5) * pow(sin(slope_1m)/0.09,1.0)"
r.colors lsfac3d_s1_1m color=gyr -e
r.colors lsfac3d_1m raster=lsfac3d_s1_1m

<div class="alert alert-block alert-info">

### Question: Which equation represents conditions when contributing area has greater impact and which models stronger influence of slope? 

...Write your response here...

Display layers and save outputs

In [ ]:
# Create Map instance
lsfac_s13_map = gj.Map(height=600, width=600, use_region=True, filename="output/lsfac_s13.png")

# Add raster layers
lsfac_s13_map.d_rast(map="lsfac3d_1m")


# Add vector layers
lsfac_s13_map.d_vect(map="elev_lid792_cont1m")

# Add map properties
lsfac_s13_map.d_legend(raster="lsfac3d_1m", at="12,50,6,10", flags="b")
lsfac_s13_map.d_barscale(at=[4,9], flags="n")

# Display map
lsfac_s13_map.show()

In [ ]:
# Create Map instance
lsfac_s10_map = gj.Map(height=600, width=600, use_region=True, filename="output/lsfac_s10.png")

# Add raster layers
lsfac_s10_map.d_rast(map="lsfac3d_s1_1m")


# Add vector layers
lsfac_s10_map.d_vect(map="elev_lid792_cont1m")

# Add map properties
lsfac_s10_map.d_legend(raster="lsfac3d_s1_1m", at="12,50,6,10", flags="b")
lsfac_s10_map.d_barscale(at=[4,9], flags="n")

# Display map
lsfac_s10_map.show()

## Compute soil detachment for spatially variable land cover

Set region to rural area and recode `landcover_1m` to `cfactor` using the r.recode module. Assign special color table and display result.

In [ ]:
%%bash

g.region rural_1m -p
r.recode landcover_1m output=cfactor_1m rules=inputs/cfac_rules.txt
r.colors map=cfactor_1m rules=inputs/cfac_color.txt

In [ ]:
# Create Map instance
cfactor_1m_map = gj.Map(height=600, width=600, use_region=True)

# Add raster layers
cfactor_1m_map.d_rast(map="cfactor_1m")

# Add map properties
cfactor_1m_map.d_legend(raster="cfactor_1m", at="12,50,6,10", flags="b")
cfactor_1m_map.d_barscale(at=[4,9], flags="n")

# Display map
cfactor_1m_map.show()

Compute the USLE3D equation using map algebra, `cfactorbare_1m` is the same as `cfactor_1m`, `cfactorgrow_1m` has landuse recoded for summer conditions.
Specify units of the raster maps using the [r.support](https://grass.osgeo.org/grass83/manuals/r.support.html) module.

In [ ]:
%%bash

r.mapcalc "soillossbare_1m = 270. * soils_Kfactor * lsfac3d_1m * cfactorbare_1m"
r.mapcalc "soillossgrow_1m = 270. * soils_Kfactor * lsfac3d_1m * cfactorgrow_1m"
r.colors soillossbare_1m rules=inputs/soilloss_color.txt
r.colors soillossgrow_1m raster=soillossbare_1m
r.support map=soillossbare_1m units="ton/(acre.year)"
r.support map=soillossgrow_1m units="ton/(acre.year)"

<div class="alert alert-block alert-info">

### Task: Compare erosion rates and distribution for winter (bare) and summer conditions. 

In [ ]:
%%bash
## Add your code here

Display the bare soil loss map

In [ ]:
# Create Map instance
soillossbare_map = gj.Map(height=600, width=600, use_region=True, filename="output/soillossbare.png")

# Add raster layers
soillossbare_map.d_rast(map="soillossbare_1m")

# Add map properties
soillossbare_map.d_legend(raster="soillossbare_1m", at="12,50,6,10", flags="b")
soillossbare_map.d_barscale(at=[4,9], flags="n")

# Display map
soillossbare_map.show()

Calculate the univariate statistics.

In [ ]:
%%bash
r.univar soillossbare_1m

Display the soil loss map during the for the summer grow season.

In [ ]:
# Create Map instance
soillosgrow_map = gj.Map(height=600, width=600, use_region=True, filename="output/soillosgrow.png")

# Add raster layers
soillosgrow_map.d_rast(map="soillossgrow_1m")

# Add map properties
soillosgrow_map.d_legend(raster="soillossgrow_1m", at="12,50,6,10", flags="b")
soillosgrow_map.d_barscale(at=[4,9], flags="n")

# Display map
soillosgrow_map.show()

Calculate the univariate statistics.

In [ ]:
%%bash
r.univar soillossgrow_1m

## Compute new DEM with erosion carved-in

In [ ]:
%%bash

r.mapcalc "elev_erodedb_1m = elev_lid792_1m-(soillossbare_1m/100.)"

Display `elev_erodedb_1m` in 3D and drape over `soillossbare_1m` as color.
To view it in 3D switch off everything except `elev_erodedb_1m`.
Drape `soillossbare_1m` as color and don't forget to set fine resolution to 1.
Use lighting from SW, z-exag at least 2.0

In [ ]:
elev_erodedb_1m_3dmap = gj.Map3D(filename="output/elev_erodedb_1m.png")
# Full list of options m.nviz.image
# https://grass.osgeo.org/grass83/manuals/m.nviz.image.html
elev_erodedb_1m_3dmap.render(
    elevation_map="elev_erodedb_1m",
    color_map="soillossbare_1m",
    perspective=20,
    height=1300,
    zexag=2,
    light_position="-1,-1,0.5",
    fringe=['ne','nw','sw','se'],
    arrow_position=[100,50],
)
elev_erodedb_1m_3dmap.overlay.d_legend(
    raster="soillossbare_1m",
    at=(40, 80, 85, 90),
    title_fontsize=12,
    fontsize=12
)

elev_erodedb_1m_3dmap.overlay.d_barscale(at=(60, 97, 87, 92))

elev_erodedb_1m_3dmap.show()

## Compute net erosion/deposition maps (using USPED)

Compute net erosion/deposition maps as divergence of sediment flow (USPED).

First compute sediment flow

In [ ]:
%%bash

r.mapcalc "sedflow_1m = 270. * soils_Kfactor * cfactorgrow_1m * flowacc_1m * sin(slope_1m)"
r.colors sedflow_1m raster=soillossbare_1m

In [ ]:
# Create Map instance
sedflow_1m_map = gj.Map(height=600, width=600, use_region=True, filename="output/sedflow_1m.png")

# Add raster layers
sedflow_1m_map.d_rast(map="sedflow_1m")

# Add map properties
sedflow_1m_map.d_legend(raster="sedflow_1m", at="12,50,6,10", flags="b")
sedflow_1m_map.d_barscale(at=[4,9], flags="n")

# Display map
sedflow_1m_map.show()

Now compute its components in x, y directions

In [ ]:
%%bash

r.mapcalc "qsx = sedflow_1m * cos(aspect_1m)"
r.mapcalc "qsy = sedflow_1m * sin(aspect_1m)"

Compute change of sediment flow in the x and y directions and then change in the direction of flow using divergence.

In [ ]:
%%bash

r.slope.aspect elevation=qsx dx=qsx_dx
r.slope.aspect elevation=qsy dy=qsy_dy
r.mapcalc "erdep = qsx_dx + qsy_dy"
r.info -r erdep
r.colors erdep rules=inputs/erdep_color.txt

In [ ]:
# Create Map instance
erdep_map = gj.Map(height=600, width=600, use_region=True, filename="output/erdep.png")

# Add raster layers
erdep_map.d_rast(map="erdep")

# Add map properties
erdep_map.d_legend(raster="erdep", at="12,50,6,10", flags="b", range="-100,100")
erdep_map.d_barscale(at=[4,9], flags="n")

# Display map
erdep_map.show()

<div class="alert alert-block alert-info">

### Task: Display elev_lid792_1m in 3D and drape over erdep as color (switch off all layers except for elev_lid792_1m).

In [ ]:
## Add your code here

## Compute summary statistics

Use [r.recode](https://grass.osgeo.org/grass76/manuals/r.recode.html) to classify erosion/deposition and [r.category](https://grass.osgeo.org/grass76/manuals/r.category.html) to add labels (stable, high erosion, etc) to individual categories: 

Example output: 

```
[...]
| #| description         |  %  | hectares |  acres  |
|-4| severe erosion . . .| 0.19|  0.101300|  0.25031|
|-3| high erosion . . . .| 1.34|  0.701600|  1.73365|
|-2| moderate erosion . .| 3.89|  2.042600|  5.04726|
|-1| low erosion . . . . |19.74| 10.366000| 25.61438|
| 0| stable . . . . . . .|61.32| 32.192000| 79.54643|
| 1| low deposition . . .| 8.40|  4.407600| 10.89118|
| 2| moderate deposition | 2.49|  1.307500|  3.23083|
| 3| high deposition . . | 1.29|  0.676900|  1.67262|
| 4| severe deposition . | 0.24|  0.126100|  0.31159|
| *|no data. . . . . . . | 1.10|  0.578400|  1.42922|
|---------------------------------------------------|
|TOTAL                   |100.00| 52.500000|129.7275|
```

In [ ]:
%%bash

r.recode erdep output=erdep_class rules=inputs/erdep_class.txt
r.category erdep_class rules=inputs/erdep_label.txt sep=:
r.report erdep_class unit=p,h,a

<div class="alert alert-block alert-info">

### Question: What are the advantages, disadvantages and risks of classifying erosion/deposition data into categories?

...Write your response here...

Compute univariate statistics: 

In [ ]:
%%bash
r.univar erdep

> Look for line with sum: **-2374.473814**

The units are tons/(acre.year), but our cells are 1 m2. Therefore we have to divide by 4046 (1 acre = 4046 m2) to get total ton per year transported from the watershed. You can use the Python shell in the Python tab do the division. 